# TODO
- [X] specify lesson plan in json format, then convert to markdown
- [ ] parallelize the lesson plan creation
- [ ] generate all activity ideas at once, so there is less repetition in the ideas (e.g., there are many small variations on the 'scavenger hunt' idea)
- [ ] try different models:
  - [X] chat model instead of LLM
  - [ ] different providers instead of OpenAI
  - [ ] local model

In [1]:
import os
import json
import time
import pprint as pp
import pandas as pd
from pydantic import BaseModel, Field
from typing import List, Dict
from dotenv import load_dotenv
# from langchain.prompts import PromptTemplate
# from langchain.llms import OpenAI

from langchain.output_parsers import PydanticOutputParser
from IPython.display import display, Markdown, JSON

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate

In [4]:
load_dotenv(override=True) # loading API key from environment

True

## Read in data with annual curriculum plan

In [5]:
GSHEET_ID = os.getenv('GSHEET_ID')
google_sheet_url = f"https://docs.google.com/spreadsheets/d/{GSHEET_ID}/export?format=csv&gid=0"
annual_plan_df = pd.read_csv(google_sheet_url)
annual_plan_df.rename(columns=lambda x: x.replace(" ", "_").lower(), inplace=True)
annual_plan_df

,month,theme_to_explore,language_arts,math,social_studies,science
0,August,Getting to Know One Another; Class Community,"Personal narratives, reading comprehension",Counting and sorting,Our classroom community; citizenship and gover...,Exploring the school environment and local eco...
1,September,"On the Map, where we’re from",My Place: Descriptive Writing; vocabulary deve...,latitutde and longitude; place value,My hometown; local history; geography,"Water cycle, rivers; Geology, caves; acquifers"
2,October,Fall; Halloween; Dia de los Muertos,Creative Writing; Listening and speaking skills,"units of measurement--length, weight, etc.; Es...",Fall celebrations around the world; elections,Animal science: Investigating Local Food Chains
3,November,Gratitude; Native Americans,Thankful Thoughts; Native American Storytelling,Addition and subtraction,Native American history; settlement of Texas,Investigating Seasonal Changes and Plants; Win...
4,December,Winter Solstice and winter holidays,book exchange; holiday cards; holiday play/skit,Snowflake Symmetry; geometry (2-D shapes),Winter Holidays Worldwide,"Physical science -- matter, energy, force, motion"
5,January,"Calendar, New Year, Goals, MLK",dreams and goals; journaling; 'I Have a Dream'...,"Time units, Calendar math, multiplication",New Year's Celebrations Around the World; Comm...,"Weather, Climate Zones, and Seasons"
6,February,"Black History, Love/Valentine's Day",African American biographies; exploring emotio...,"Fractions, Division; Number Patterns",African American Achievements; Civil Rights Mo...,"The human body, heart health and exercise"
7,March,Spring; Easter; Ramadan,Springtime poetry; nature observation,Patterns and Geometry; Geometry in Islamic Art,"Cultural Spring Celebrations -- Easter, Ramadan",Life Cycles of Plants and Animals (frogs and b...
8,April,Space; Passover,"Space stories, science fiction; constellation ...",Patterns and algebraic thinking; math in the s...,Passover origins; History of astronomy,Exploring the Solar System and Celestial Bodies
9,May,Ocean Life,Ocean vocabulary; sea creatures; sea stories,Problem-solving and mathematical reasoning; oc...,Coastal Ecosystems; exploration and navigation,"Ocean zones, Marine Life and Habitats"


## Instantiate the Chat Model

In [6]:
# model_name = "text-davinci-003"
model_name = "gpt-3.5-turbo"
temperature = 0.1
# model = OpenAI(model_name=model_name, temperature=temperature, max_tokens=3000)

In [7]:
chat_model = ChatOpenAI(model_name=model_name, temperature=temperature, max_tokens=3000)

In [8]:
chat_model = ChatOpenAI(model_name=model_name, temperature=temperature, max_tokens=3000)

# Create Prompt Templates

In [9]:
class ActivityIdea(BaseModel):
    name: str
    description: str

class ActivityIdeasOutput(BaseModel):
    ideas: List[ActivityIdea]
# Set up a parser + inject instructions into the prompt template.
activity_ideas_parser = PydanticOutputParser(pydantic_object=ActivityIdeasOutput)

In [10]:
idea_human_prompt = HumanMessagePromptTemplate.from_template(
    template="""Answer the user query.
    {format_instructions}
    You are a curriculum development expert and I am a teacher of second graders at a micro-school, with seven students of varying abilities.
    I want you to generate {num_ideas} activity ideas for the month of {month}. Each activity could also be thought of as a lesson where the kids are learning valuable skills.
    I have a theme for the month, which is {theme}.
    The academic subject is {subject} and the topic is {topic}.

    Here are some additional guidelines:
    - The microschool  stresses the importance of outdoor education and student-led development, but it also wants to ensure high academic standards, so your lesson plans should be a mix of exploration and drilling the fundamentals. 
    - Try to be creative in your ideas. Do not use the idea of a scavenger hunt, because it has been overdone.
    - Try to work in phonics work and literacy development into the language art activities.
    - Try to work in arithmetic and numeracy development into the math activities.
    - Each activity idea should be able to be accomplished in 30 mins-3 hours.
    - Each activity should relate directly to skills and concepts from the academic subject of {subject}.
    - I only need you to give me the activity names and a brief description of each idea.
    - The description should only be sentence or two and should not simply repeat the theme or the topic. It should describe the nature of the activity.
    - Keep in mind these are second-graders who are not ready for advanced concepts like decimals.
    """
)

In [11]:
idea_chat_prompt_template = ChatPromptTemplate(
    messages=[idea_human_prompt],
    input_variables=["num_ideas", "month", "theme", "subject", "topic"],
    partial_variables={"format_instructions": activity_ideas_parser.get_format_instructions()}
)

In [12]:
print(activity_ideas_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ideas": {"title": "Ideas", "type": "array", "items": {"$ref": "#/definitions/ActivityIdea"}}}, "required": ["ideas"], "definitions": {"ActivityIdea": {"title": "ActivityIdea", "type": "object", "properties": {"name": {"title": "Name", "type": "string"}, "description": {"title": "Description", "type": "string"}}, "required": ["name", "description"]}}}
```


In [13]:
class LessonPlanOutput(BaseModel):
    time_estimate: str
    list_of_materials: List[str]
    procedure: List[str]
    additional_notes: List[str]
    differentiation_strategies: List[str]

# Set up a parser + inject instructions into the prompt template.
lesson_plan_parser = PydanticOutputParser(pydantic_object=LessonPlanOutput)

In [14]:
lesson_plan_human_prompt = HumanMessagePromptTemplate.from_template(
    template="""Answer the user query.
    {format_instructions}
    You are a curriculum development expert and i am a teacher of second graders at a micro-school, with seven students of varying abilities.
    I want you to generate a *detailed* lesson plan.
    The name of the lesson is {activity_name}, and the short description is: {activity_description}

    Additional guidelines for the lesson plan:
    - The microschool  stresses the importance of outdoor education and student-led development, but it also wants to ensure high academic standards.
    - Ensure that the lesson uses and develops skills relevant to the academic subject: {subject}
    - Try to make the best estimate of the total amount of time the activity will take. Be generous because these are young kids and they take a while!
    - The 'additional notes' specifies the actual content mentioned in the 'procedure' (e.g., interview questions, puzzle clues, specific writing prompts, etc.), so the teacher does not need to do additional brainstorming to execute the lesson.
    - The 'differentiation strategies' provides modifications for students who need more support, for students who need more challenge, and for neurodivergent students. This information should not appear elsewhere in your response.
    """
)

In [15]:
lesson_plan_chat_prompt_template = ChatPromptTemplate(
    messages=[lesson_plan_human_prompt],
    input_variables=["activity_name", "activity_description", "subject"],
    partial_variables={"format_instructions": lesson_plan_parser.get_format_instructions()}
)

In [16]:
print(lesson_plan_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"time_estimate": {"title": "Time Estimate", "type": "string"}, "list_of_materials": {"title": "List Of Materials", "type": "array", "items": {"type": "string"}}, "procedure": {"title": "Procedure", "type": "array", "items": {"type": "string"}}, "additional_notes": {"title": "Additional Notes", "type": "array", "items": {"type": "string"}}, "differentiation_strategies": {"title": "Differentiation Strategies", "type": "array", "items": {"type": "string"}}}, "required": ["time_estimate", "list_of_materials", "procedure", "addition

# Test on Single Record

In [17]:
num_plans = 5

In [18]:
month = "September"
subject = 'math'

In [19]:
month_record = annual_plan_df.query("month == @month").to_dict(orient='records')[0]

In [20]:
theme = month_record['theme_to_explore']
topic = month_record[subject]

In [21]:
idea_prompt = idea_chat_prompt_template.format_prompt(
    num_ideas=num_plans,
    month=month,
    theme=theme,
    subject=subject, 
    topic=topic 
)

In [22]:
idea_messages = idea_prompt.to_messages()

In [23]:
idea_messages

[HumanMessage(content='Answer the user query.\n    The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"ideas": {"title": "Ideas", "type": "array", "items": {"$ref": "#/definitions/ActivityIdea"}}}, "required": ["ideas"], "definitions": {"ActivityIdea": {"title": "ActivityIdea", "type": "object", "properties": {"name": {"title": "Name", "type": "string"}, "description": {"title": "Description", "type": "string"}}, "required": ["name", "description"]}}}\n```\n    You are a curriculum development expert and I am a teacher of second graders at a micro-school, with seven stud

In [24]:
idea_response = chat_model(idea_messages)

In [25]:
ideas_parsed = activity_ideas_parser.parse(idea_response.content)

In [26]:
pp.pprint(ideas_parsed.dict())

{'ideas': [{'description': 'Students will create their own maps using basic '
                           'shapes and colors, practicing spatial awareness '
                           'and understanding of cardinal directions.',
            'name': 'Map Art'},
           {'description': 'Students will participate in a physical activity '
                           'where they move along a number line, reinforcing '
                           'their understanding of place value and number '
                           'sequencing.',
            'name': 'Number Line Race'},
           {'description': 'Students will play a game of bingo using latitude '
                           'and longitude coordinates, improving their ability '
                           'to locate and identify points on a map.',
            'name': 'Latitude and Longitude Bingo'},
           {'description': 'Students will create and share stories that '
                           'incorporate math concepts, enhancing 

In [27]:
idea = ideas_parsed.ideas[0]

In [28]:
idea_name = idea.name
idea_description = idea.description

In [29]:
print(f"{idea_name}: {idea_description}")

Map Art: Students will create their own maps using basic shapes and colors, practicing spatial awareness and understanding of cardinal directions.


In [30]:
lesson_plan_prompt_value = lesson_plan_chat_prompt_template.format_prompt(
    activity_name=idea_name,
    activity_description=idea_description,
    subject=subject
)

In [31]:
lp_messages = lesson_plan_prompt_value.to_messages()

In [32]:
lp_response = chat_model(lp_messages)

In [33]:
lp_parsed = lesson_plan_parser.parse(lp_response.content)

In [34]:
lp_dict = lp_parsed.dict()

In [35]:
full_lp = {k: v for k, v in idea.dict().items()}
full_lp.update(lp_dict)

In [36]:
lp_dict['additional_notes']

['Encourage creativity and imagination when drawing the maps.',
 'Provide guidance and support to students who may struggle with drawing or labeling.',
 'For students who finish early, encourage them to add additional details or features to their maps.',
 'For students who need more support, provide pre-drawn templates for them to use as a starting point.',
 'For students who need more challenge, encourage them to include a key or legend on their maps to represent different symbols or features.',
 'For neurodivergent students, provide visual aids or step-by-step instructions to help them with the drawing and labeling process.']

## Define Markdown Helper Functions

In [37]:
def make_ordered_list_markdown(lst):
    markdown_list = [f"{idx}. {val}" for idx, val in enumerate(lst, start=1)]
    return '\n'.join(markdown_list)

In [38]:
def make_list_markdown(lst):
    return "- " + "\n- ".join(lst)

In [39]:
def convert_lesson_plan_to_markdown(lesson_plan_dict):
    res = []
    res.append(f"## {lesson_plan_dict['name']}")
    res.append(lesson_plan_dict['description'])
    res.append(f"\n### Time estimate: {lesson_plan_dict['time_estimate']}")
    res.append(f"\n### Materials")
    res.append(make_list_markdown(lesson_plan_dict["list_of_materials"]))
    res.append(f"\n### Procedure")
    res.append(make_ordered_list_markdown(lesson_plan_dict['procedure']))
    res.append(f"\n### Additional notes")
    res.append(make_list_markdown(lesson_plan_dict['additional_notes']))
    res.append(f"\n### Differentiation strategies")
    res.append(make_list_markdown(lesson_plan_dict['differentiation_strategies']))
    
    return "\n".join(res)
    

In [41]:
md = convert_lesson_plan_to_markdown(full_lp)

In [42]:
display(Markdown(md))

## Map Art
Students will create their own maps using basic shapes and colors, practicing spatial awareness and understanding of cardinal directions.

### Time estimate: 60 minutes

### Materials
- Paper
- Pencils
- Crayons
- Rulers
- Scissors

### Procedure
1. Start by discussing maps with the students. Talk about what maps are, why they are important, and how they can be used.
2. Explain the concept of cardinal directions (north, south, east, west) and how they are represented on a map.
3. Provide each student with a piece of paper and ask them to draw a basic map of their choosing. Encourage them to include different shapes and colors to represent different features on their map.
4. Once the students have finished drawing their maps, have them label the cardinal directions on their maps.
5. After labeling the cardinal directions, ask the students to share their maps with the class. Have them explain the different features on their maps and how they represent real-world locations.
6. As a math activity, ask the students to measure the distances between different features on their maps using rulers. Have them record the measurements on their maps.
7. Finally, have a class discussion about the importance of spatial awareness and how it relates to math. Ask the students to reflect on how they used math skills during the activity.

### Additional notes
- Encourage creativity and imagination when drawing the maps.
- Provide guidance and support to students who may struggle with drawing or labeling.
- For students who finish early, encourage them to add additional details or features to their maps.
- For students who need more support, provide pre-drawn templates for them to use as a starting point.
- For students who need more challenge, encourage them to include a key or legend on their maps to represent different symbols or features.
- For neurodivergent students, provide visual aids or step-by-step instructions to help them with the drawing and labeling process.

### Differentiation strategies
- Provide pre-drawn templates for students who need more support.
- Encourage students who need more challenge to include a key or legend on their maps.
- Provide visual aids or step-by-step instructions for neurodivergent students.

# Define Wrapper Functions

In [43]:
def prompt_wrapper(prompt_template, parser, model, max_retries=5, **kwargs):
    prompt_value = prompt_template.format_prompt(**kwargs)
    messages = prompt_value.to_messages()
    for attempt in range(max_retries + 1):  #sometimes the parser fails inexplicably
        try:
            output = model(messages)
            parsed_output = parser.parse(output.content)
            break
        except Exception as e:
            if attempt < max_retries:
                # print(e)
                print(f"Attempt {attempt + 1} failed. Retrying...")
                time.sleep(2)  # Wait before retrying
            else:
                print(f"API call failed after {max_retries} attempts. Skipping.")
    return parsed_output

In [44]:
activity_ideas_parsed = prompt_wrapper(  # Generate activity ideas for each month-subject combo
                prompt_template=idea_chat_prompt_template, 
                parser=activity_ideas_parser,
                model=chat_model,
                num_ideas=num_plans,
                month=month,
                theme=theme,
                subject=subject, 
                topic=topic
            )

In [45]:
pp.pprint(activity_ideas_parsed.dict())

{'ideas': [{'description': 'Students will create their own maps using basic '
                           'shapes and colors, practicing spatial awareness '
                           'and understanding of cardinal directions.',
            'name': 'Map Art'},
           {'description': 'Students will use a giant number line to '
                           'physically move along and practice addition and '
                           'subtraction, reinforcing place value concepts.',
            'name': 'Number Line Journey'},
           {'description': 'Students will solve latitude and longitude puzzles '
                           'to locate hidden treasures on a map, enhancing '
                           'their understanding of coordinates and map reading '
                           'skills.',
            'name': 'Latitude and Longitude Scramble'},
           {'description': 'Students will listen to a story about a journey '
                           'and solve math problems related

In [47]:
def wrapper(annual_plan_df, activity_ideas_model, lesson_plan_model, num_plans=5, months=None):
    res = {}
    if months is None:
        months = annual_plan_df['month'].values
    df = annual_plan_df.query('month.isin(@months)')
    for idx, row in df.head(3).iterrows(): # first iterate through the months in the table
        month = row["month"]
        theme = row["theme_to_explore"]
        print(f"# {month}")
        markdown_lst = []
        markdown_lst.append(f"# {month}")
        markdown_lst.append(f"## Theme: {theme}")
        res[month] = dict()
        subjects = df.columns[2:].values
        for subject in subjects:  # for each month, iterate through the academic subjects
            pretty_subject = subject.replace("_", " ").title()
            markdown_lst.append(f"\n# {pretty_subject}")
            print(f"- {pretty_subject}")
            topic = row[subject]
            activity_ideas_parsed = prompt_wrapper(  # Generate activity ideas for each month-subject combo
                prompt_template=idea_chat_prompt_template, 
                parser=activity_ideas_parser,
                model=activity_ideas_model,
                num_ideas=num_plans,
                month=month,
                theme=theme,
                subject=subject, 
                topic=topic
            )
            print("Generated ideas")
            res[month][subject] = activity_ideas_parsed.dict()
            
            for idea in res[month][subject]["ideas"]: #  Generate lesson plans for each idea
                activity_name = idea["name"]
                activity_description = idea["description"]
                lesson_plan = prompt_wrapper(
                    prompt_template=lesson_plan_chat_prompt_template, 
                    parser=lesson_plan_parser,
                    model=lesson_plan_model,
                    activity_name=activity_name, 
                    activity_description=activity_description, 
                    subject=subject
                )
                idea.update(lesson_plan)
                lesson_plan_markdown = convert_lesson_plan_to_markdown(idea)
                markdown_lst.append(f"\n{lesson_plan_markdown}")
                markdown_string = "\n".join(markdown_lst)
            print("Generated lesson plans")
                
        # Write the output of each month to a Markdown file
        output_file_path = f'../output/{idx + 1} {month} Lesson Plans.md'
        with open(output_file_path, "w") as output_file:
            output_file.write(markdown_string)

        print(f"Markdown output written to '{output_file_path}'.")
        print("*" * 50, "\n")
        # break
    
    # Write full results to a JSON file
    full_result_output_path = "../output/lesson_plan.json"
    with open(full_result_output_path , "w") as json_file:
        json.dump(res, json_file)
    print(f"JSON output written to '{full_result_output_path }'.")
    return res

# Run

In [48]:
annual_plan_df.head()

,month,theme_to_explore,language_arts,math,social_studies,science
0,August,Getting to Know One Another; Class Community,"Personal narratives, reading comprehension",Counting and sorting,Our classroom community; citizenship and gover...,Exploring the school environment and local eco...
1,September,"On the Map, where we’re from",My Place: Descriptive Writing; vocabulary deve...,latitutde and longitude; place value,My hometown; local history; geography,"Water cycle, rivers; Geology, caves; acquifers"
2,October,Fall; Halloween; Dia de los Muertos,Creative Writing; Listening and speaking skills,"units of measurement--length, weight, etc.; Es...",Fall celebrations around the world; elections,Animal science: Investigating Local Food Chains
3,November,Gratitude; Native Americans,Thankful Thoughts; Native American Storytelling,Addition and subtraction,Native American history; settlement of Texas,Investigating Seasonal Changes and Plants; Win...
4,December,Winter Solstice and winter holidays,book exchange; holiday cards; holiday play/skit,Snowflake Symmetry; geometry (2-D shapes),Winter Holidays Worldwide,"Physical science -- matter, energy, force, motion"


In [49]:
lesson_plan_model = ChatOpenAI(model_name=model_name, temperature=.2, max_tokens=3000)

In [50]:
activity_ideas_model = ChatOpenAI(model_name=model_name, temperature=.8, max_tokens=3000)

In [ ]:
lesson_plans = wrapper(annual_plan_df, activity_ideas_model, lesson_plan_model, months=["September", "October"])

# September
- Language Arts
Generated ideas
Generated lesson plans
- Math
Generated ideas
Generated lesson plans
- Social Studies
Generated ideas
Generated lesson plans
- Science
Generated ideas
Generated lesson plans
Markdown output written to '../output/2 September Lesson Plans.md'.
************************************************** 

# October
- Language Arts
Generated ideas
Generated lesson plans
- Math
Generated ideas
Generated lesson plans
- Social Studies
Generated ideas
Generated lesson plans
- Science
Generated ideas


# View Results

In [ ]:
for idea in lesson_plans['September']['language_arts']['ideas']:
    display(Markdown(f"**{idea['name']}**: {idea['description']}"))

In [ ]:
display(JSON(lesson_plans))

In [ ]:
display(JSON(lesson_plans))